# Dataset Preparation

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Parameters
n_weeks = 30
n_products = 20
n_stores = 10
dates = pd.date_range(start="2025-01-01", periods=n_weeks, freq="W")

products = [f"product_{i+1}" for i in range(n_products)]
stores = [f"store_{i+1}" for i in range(n_stores)]

data = []

# Generate demand for each (store, product)
for store in stores:
    for product in products:
        base = np.random.randint(30, 100)
        trend = np.random.uniform(0.5, 3.0)
        seasonality_pattern = np.sin(np.linspace(0, 3 * np.pi, n_weeks)) * np.random.uniform(5, 20)

        # Add random seasonal behavior with type-based noise
        noise_type = np.random.choice(["normal", "poisson", "exponential"])
        if noise_type == "normal":
            noise = np.random.normal(0, 10, n_weeks)
        elif noise_type == "poisson":
            noise = np.random.poisson(3, n_weeks) - 3
        else:  # exponential
            noise = np.random.exponential(5, n_weeks) - 5

        demand = base + trend * np.arange(n_weeks) + seasonality_pattern + noise
        demand = np.maximum(0, demand).astype(int)  # no negatives

        for week, value in zip(dates, demand):
            data.append({
                'week': week,
                'store': store,
                'product': product,
                'demand': value
            })

df_forecast = pd.DataFrame(data)
df_forecast.to_csv("demand_data.csv", index=False)

In [ ]:
df_forecast

,week,store,product,demand
0,2025-01-05,store_1,product_1,69
1,2025-01-12,store_1,product_1,110
2,2025-01-19,store_1,product_1,95
3,2025-01-26,store_1,product_1,93
4,2025-02-02,store_1,product_1,99
...,...,...,...,...
5995,2025-06-29,store_10,product_20,86
5996,2025-07-06,store_10,product_20,88
5997,2025-07-13,store_10,product_20,90
5998,2025-07-20,store_10,product_20,91


# AD Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Function to test stationarity
def adf_test(series):
    result = adfuller(series, autolag='AIC')
    p_value = result[1]
    return p_value < 0.05  # Stationary if p-value < 0.05

# Run ADF test on all 200 series
adf_results = []

for store in df_forecast['store'].unique():
    for product in df_forecast['product'].unique():
        series = df_forecast[(df_forecast['store'] == store) &
                           (df_forecast['product'] == product)].sort_values(by='week')['demand'].reset_index(drop=True)
        is_stationary = adf_test(series)
        adf_results.append({
            'store': store,
            'product': product,
            'is_stationary': is_stationary
        })

df_adf = pd.DataFrame(adf_results)
df_adf.head()

,store,product,is_stationary
0,store_1,product_1,False
1,store_1,product_2,False
2,store_1,product_3,False
3,store_1,product_4,False
4,store_1,product_5,False


In [ ]:
df_adf[df_adf["is_stationary"] == True].sum()

,0
store,store_2store_3store_3store_3store_7store_9
product,product_7product_2product_3product_11product_1...
is_stationary,6


In [ ]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Entities
plants = [f"plant_{i+1}" for i in range(3)]
dcs = [f"dc_{i+1}" for i in range(5)]
stores = [f"store_{i+1}" for i in range(10)]
products = [f"product_{i+1}" for i in range(20)]


In [ ]:
plant_dc_cost = []

for plant in plants:
    for dc in dcs:
        for product in products:
            cost = round(np.random.uniform(1.0, 5.0), 2)
            plant_dc_cost.append({
                "from_plant": plant,
                "to_dc": dc,
                "product": product,
                "cost_per_unit": cost
            })

df_plant_dc_cost = pd.DataFrame(plant_dc_cost)

df_plant_dc_cost.to_csv("plant_dc_cost.csv", index=False)


In [ ]:
df_plant_dc_cost

,from_plant,to_dc,product,cost_per_unit
0,plant_1,dc_1,product_1,2.50
1,plant_1,dc_1,product_2,4.80
2,plant_1,dc_1,product_3,3.93
3,plant_1,dc_1,product_4,3.39
4,plant_1,dc_1,product_5,1.62
...,...,...,...,...
295,plant_3,dc_5,product_16,3.09
296,plant_3,dc_5,product_17,4.08
297,plant_3,dc_5,product_18,1.86
298,plant_3,dc_5,product_19,3.49


In [ ]:
dc_store_cost = []

for dc in dcs:
    for store in stores:
        for product in products:
            cost = round(np.random.uniform(0.5, 3.0), 2)
            dc_store_cost.append({
                "from_dc": dc,
                "to_store": store,
                "product": product,
                "cost_per_unit": cost
            })

df_dc_store_cost = pd.DataFrame(dc_store_cost)
df_dc_store_cost.to_csv("dc_store_cost.csv", index=False)

In [ ]:
df_dc_store_cost

,from_dc,to_store,product,cost_per_unit
0,dc_1,store_1,product_1,0.63
1,dc_1,store_1,product_2,1.83
2,dc_1,store_1,product_3,1.85
3,dc_1,store_1,product_4,2.09
4,dc_1,store_1,product_5,2.32
...,...,...,...,...
995,dc_5,store_10,product_16,1.00
996,dc_5,store_10,product_17,0.92
997,dc_5,store_10,product_18,0.76
998,dc_5,store_10,product_19,2.09


# Due to Limited compuational power predicting for 10 stores and 20 products each


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

# ----- Config -----
# Update to 10 stores and 20 products
stores = [f'store_{i+1}' for i in range(10)]
products = [f'product_{i+1}' for i in range(20)]
forecast_horizon = [26, 27, 28, 29, 30]

# Filter data
demo_df = df_forecast [(df_forecast ['store'].isin(stores)) & (df_forecast ['product'].isin(products))].copy()
demo_df['week'] = pd.to_datetime(demo_df['week'])

# ----- Helper: Forecasting function -----
def forecast_models(train_series):
    forecasts = {}
    # Exponential Smoothing
    try:
        model_es = ExponentialSmoothing(train_series, trend='add', seasonal=None).fit()
        forecasts['exp'] = model_es.forecast(1)[0]
    except:
        forecasts['exp'] = np.nan

    # ARIMA
    try:
        model_arima = ARIMA(train_series, order=(2,1,2)).fit()
        forecasts['arima'] = model_arima.forecast(1)[0]
    except:
        forecasts['arima'] = np.nan

    # SARIMA
    try:
        model_sarima = SARIMAX(train_series, order=(1,1,1), seasonal_order=(1,1,1,7)).fit(disp=False)
        forecasts['sarima'] = model_sarima.forecast(1)[0]
    except:
        forecasts['sarima'] = np.nan

    return forecasts

# ----- Step 1: Forecast table -----
forecast_records = []

for store in stores: # Loop through stores
    for product in products: # Loop through products
        ts = demo_df[(demo_df['store'] == store) & (demo_df['product'] == product)].sort_values(by='week')
        ts = ts.set_index('week')['demand']

        # Iterate through the indices corresponding to the forecast horizon
        for i in range(25, 30): # Indices 25 to 29 correspond to weeks 26 to 30
            train = ts.iloc[i - 25:i] # Use iloc with the correct index
            real = ts.iloc[i]         # Use iloc with the correct index
            forecasts = forecast_models(train)

            forecast_records.append({
                'store': store,
                'product': product,
                'week': ts.index[i], # Use the correct index to get the week
                'ARIMA': forecasts['arima'],
                'SARIMA': forecasts['sarima'],
                'Exponential': forecasts['exp'],
                'Real': real
            })

df_preds = pd.DataFrame(forecast_records)

In [ ]:
df_preds_final = df_preds[['store', 'product', 'week', 'ARIMA', 'SARIMA', 'Exponential', 'Real']]

In [ ]:
print("Forecasts:")
display(df_preds_final)

Forecasts:


,store,product,week,ARIMA,SARIMA,Exponential,Real
0,store_1,product_1,2025-06-29,162.820623,176.961677,167.214420,174
1,store_1,product_1,2025-07-06,171.422481,144.498810,173.834184,167
2,store_1,product_1,2025-07-13,172.695290,178.914522,173.497039,168
3,store_1,product_1,2025-07-20,171.402836,174.631602,173.555266,159
4,store_1,product_1,2025-07-27,163.720173,169.063128,167.591682,156
...,...,...,...,...,...,...,...
995,store_10,product_20,2025-06-29,88.687422,88.399616,88.023548,86
996,store_10,product_20,2025-07-06,88.362955,88.699089,87.842628,88
997,store_10,product_20,2025-07-13,87.659590,89.296119,88.671342,90
998,store_10,product_20,2025-07-20,90.958935,91.066259,90.736539,91


In [ ]:
DF = df_preds_final[['store','product','week','ARIMA','SARIMA','Exponential','Real']]

In [ ]:
DF

,store,product,week,ARIMA,SARIMA,Exponential,Real
0,store_1,product_1,2025-06-29,162.820623,176.961677,167.214420,174
1,store_1,product_1,2025-07-06,171.422481,144.498810,173.834184,167
2,store_1,product_1,2025-07-13,172.695290,178.914522,173.497039,168
3,store_1,product_1,2025-07-20,171.402836,174.631602,173.555266,159
4,store_1,product_1,2025-07-27,163.720173,169.063128,167.591682,156
...,...,...,...,...,...,...,...
995,store_10,product_20,2025-06-29,88.687422,88.399616,88.023548,86
996,store_10,product_20,2025-07-06,88.362955,88.699089,87.842628,88
997,store_10,product_20,2025-07-13,87.659590,89.296119,88.671342,90
998,store_10,product_20,2025-07-20,90.958935,91.066259,90.736539,91


In [ ]:
abs(DF['ARIMA'] - DF['Real']).sum()

np.float64(6583.878305231352)

In [ ]:
abs(DF['Exponential'] - DF['Real']).sum()

np.float64(6243.55206679814)

In [ ]:
abs(DF['SARIMA'] - DF['Real']).sum()

np.float64(7925.261899506326)

# Building the Ensemble Model

In [ ]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

# Prepare data for regression
# Use df_final_table and select relevant columns, dropping rows with NaNs in features or target
regression_df = DF[['ARIMA', 'SARIMA', 'Exponential', 'Real']].dropna()

X = regression_df[['ARIMA', 'SARIMA', 'Exponential']]
y = regression_df['Real']

# Fit Linear Regression model
model = LinearRegression()
model.fit(X, y)

# Get the weights (coefficients)
weights = model.coef_

# Display the weights
print("Learned weights for the linear regression ensemble:")
print(f"ARIMA weight: {weights[0]}")
print(f"SARIMA weight: {weights[1]}")
print(f"Exponential weight: {weights[2]}")


# Optionally, display the intercept
print(f"Intercept: {model.intercept_}")

Learned weights for the linear regression ensemble:
ARIMA weight: 0.048388933097011456
SARIMA weight: 0.06579374887661127
Exponential weight: 0.8515489538363905
Intercept: 1.6793511474714933


In [ ]:
DF['Ensemble_Model'] = DF['ARIMA']*weights[0] + DF['SARIMA']*weights[1] + DF['Exponential']*weights[2] + model.intercept_

In [ ]:
DF

,store,product,week,ARIMA,SARIMA,Exponential,Real,Ensemble_Model
0,store_1,product_1,2025-06-29,162.820623,176.961677,167.214420,174,163.592304
1,store_1,product_1,2025-07-06,171.422481,144.498810,173.834184,167,167.509738
2,store_1,product_1,2025-07-13,172.695290,178.914522,173.497039,168,169.548571
3,store_1,product_1,2025-07-20,171.402836,174.631602,173.555266,159,169.253824
4,store_1,product_1,2025-07-27,163.720173,169.063128,167.591682,156,163.437414
...,...,...,...,...,...,...,...,...
995,store_10,product_20,2025-06-29,88.687422,88.399616,88.023548,86,86.743344
996,store_10,product_20,2025-07-06,88.362955,88.699089,87.842628,88,86.593284
997,store_10,product_20,2025-07-13,87.659590,89.296119,88.671342,90,87.304220
998,store_10,product_20,2025-07-20,90.958935,91.066259,90.736539,91,89.338952


In [ ]:
ensemble_error = abs(DF['Ensemble_Model'] - DF['Real']).sum()

In [ ]:
avg_error = (abs(DF['ARIMA'] - DF['Real']).sum() + abs(DF['Exponential'] - DF['Real']).sum() + abs(DF['SARIMA'] - DF['Real']).sum())/3

In [ ]:
avg_error

np.float64(6917.564090511939)

In [ ]:
ensemble_error

np.float64(5800.6650656145)

In [ ]:
improvement = ((avg_error - ensemble_error)/avg_error)*100

In [ ]:
print(improvement)

16.145842818129665


In [ ]:
print(improvement)

# Improvement in Ensemble Model is 16.14%

# OPTIMISATION

In [ ]:
%pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 91.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pulp

# Load datasets
df_demand = pd.read_csv('demand_data.csv')
df_plant_dc = pd.read_csv('plant_dc_cost.csv')
df_dc_store = pd.read_csv('dc_store_cost.csv')

# Unique sets
plants = df_plant_dc['from_plant'].unique()
dcs = df_plant_dc['to_dc'].unique()
stores = df_demand['store'].unique()
products = df_demand['product'].unique()
weeks = df_demand['week'].unique()

# Cost dictionaries
cost_p2dc = {(row['from_plant'], row['to_dc'], row['product']): row['cost_per_unit']
             for _, row in df_plant_dc.iterrows()}

cost_dc2s = {(row['from_dc'], row['to_store'], row['product']): row['cost_per_unit']
             for _, row in df_dc_store.iterrows()}

# Output lists
results_plant_dc = []
results_dc_store = []

# Solve per week
for week in weeks:
    week_data = df_demand[df_demand['week'] == week]

    model = pulp.LpProblem(f"Transport_Week_{week}", pulp.LpMinimize)

    # Decision variables
    x_vars = pulp.LpVariable.dicts("x", [(i, j, k) for i in plants for j in dcs for k in products], lowBound=0, cat='Continuous')
    y_vars = pulp.LpVariable.dicts("y", [(j, s, k) for j in dcs for s in stores for k in products], lowBound=0, cat='Continuous')

    # Objective
    model += pulp.lpSum([x_vars[i, j, k] * cost_p2dc.get((i, j, k), 0)
                         for i in plants for j in dcs for k in products]) + \
             pulp.lpSum([y_vars[j, s, k] * cost_dc2s.get((j, s, k), 0)
                         for j in dcs for s in stores for k in products])

    # Demand constraints
    for _, row in week_data.iterrows():
        s, k, d = row['store'], row['product'], row['demand']
        model += pulp.lpSum([y_vars[j, s, k] for j in dcs]) == d

    # Flow conservation
    for j in dcs:
        for k in products:
            model += pulp.lpSum([x_vars[i, j, k] for i in plants]) == pulp.lpSum([y_vars[j, s, k] for s in stores])

    # Solve
    model.solve()

    # Store results
    for (i, j, k), var in x_vars.items():
        if var.varValue > 0:
            results_plant_dc.append({
                'week': week,
                'from_plant': i,
                'to_dc': j,
                'product': k,
                'quantity': var.varValue
            })

    for (j, s, k), var in y_vars.items():
        if var.varValue > 0:
            results_dc_store.append({
                'week': week,
                'from_dc': j,
                'to_store': s,
                'product': k,
                'quantity': var.varValue
            })

# Convert to DataFrame and save
pd.DataFrame(results_plant_dc).to_csv('weekly_transport_paths_plant_dc.csv', index=False)
pd.DataFrame(results_dc_store).to_csv('weekly_transport_paths_dc_store.csv', index=False)


In [ ]:
plant_dc = pd.read_csv("/content/weekly_transport_paths_dc_store.csv")

In [ ]:
plant_dc

,week,from_dc,to_store,product,quantity
0,2025-01-05,dc_1,store_1,product_1,69.0
1,2025-01-05,dc_1,store_1,product_7,43.0
2,2025-01-05,dc_1,store_1,product_10,45.0
3,2025-01-05,dc_1,store_1,product_11,47.0
4,2025-01-05,dc_1,store_1,product_18,53.0
...,...,...,...,...,...
5995,2025-07-27,dc_5,store_9,product_6,135.0
5996,2025-07-27,dc_5,store_10,product_4,117.0
5997,2025-07-27,dc_5,store_10,product_5,67.0
5998,2025-07-27,dc_5,store_10,product_11,112.0


In [ ]:
dc_store = pd.read_csv("/content/weekly_transport_paths_plant_dc.csv")

In [ ]:
dc_store

,week,from_plant,to_dc,product,quantity
0,2025-01-05,plant_1,dc_1,product_5,32.0
1,2025-01-05,plant_1,dc_1,product_7,404.0
2,2025-01-05,plant_1,dc_1,product_11,421.0
3,2025-01-05,plant_1,dc_1,product_14,190.0
4,2025-01-05,plant_1,dc_1,product_15,270.0
...,...,...,...,...,...
1975,2025-07-27,plant_3,dc_5,product_11,274.0
1976,2025-07-27,plant_3,dc_5,product_12,793.0
1977,2025-07-27,plant_3,dc_5,product_14,131.0
1978,2025-07-27,plant_3,dc_5,product_15,294.0
